In [36]:
# Import packages
import pandas as pd
import numpy as np
import datetime as dt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import networkx as nx

# Reading and merging tables

In [37]:
# Import order product dataset
df = pd.read_csv('order_products.csv', usecols=['order_id', 'product_id'])
df.head()

,order_id,product_id
0,10,24
1,10,83
2,10,16
3,10,24
4,10,83


In [38]:
#check for null values
df.isnull().sum()

order_id      0
product_id    0
dtype: int64

In [39]:
# Import product label mapping
products = pd.read_csv('products.csv')
products.head()

,product_id,department_id,product_name
0,61,19,cookies cakes
1,104,13,spices seasonings
2,94,7,tea
3,38,1,frozen meals
4,5,13,marinades meat preparation


In [40]:
#check for null values
products.isnull().sum()

product_id       0
department_id    0
product_name     0
dtype: int64

In [41]:
# Merge product labels
df = pd.merge(df, products, how='left', on='product_id').\
    drop(["product_id", "department_id"], axis=1)
df.head()

,order_id,product_name
0,10,fresh fruits
1,10,fresh vegetables
2,10,fresh herbs
3,10,fresh fruits
4,10,fresh vegetables


In [43]:
# Merge product labels
df = pd.merge(df, products, how='left', on='product_id').\
    drop(["product_id", "department_id"], axis=1)
df.head()

KeyError: 'product_id'

# Data exploration

In [86]:
# how many departments
products['department_id'].unique()

array([19, 13,  7,  1, 11, 16, 17, 18, 12,  9,  8, 14, 15,  4, 21,  6, 20,
        5,  3,  2, 10], dtype=int64)

In [44]:
#check the average amount of items per order
count_df = df.groupby('order_id').agg(['count'])
total_orders = len(count_df)
mean_number_item_perorder=count_df['product_name']['count'].sum()/total_orders
mean_number_item_perorder

10.097505

In [45]:
# Get modes and unique values
df.astype(object).describe()

,order_id,product_name
count,2019501,2019501
unique,200000,134
top,790903,fresh fruits
freq,137,226039


In [46]:
# Check product frequencies
df['product_name'].value_counts().head(20)

fresh fruits                     226039
fresh vegetables                 212611
packaged vegetables fruits       109596
yogurt                            90751
packaged cheese                   61502
milk                              55150
water seltzer sparkling water     52564
chips pretzels                    45306
soy lactosefree                   39389
bread                             36381
refrigerated                      35893
frozen produce                    32432
ice cream ice                     31323
energy granola bars               28639
crackers                          28574
eggs                              27986
lunch meat                        24470
frozen meals                      24449
cereal                            23754
baby food formula                 23355
Name: product_name, dtype: int64

In [47]:
# Pivot the data - lines as orders and products as columns
pt = pd.pivot_table(df, index='order_id', columns='product_name', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
pt.head()

product_name,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
order_id,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# Check in how many orders was product fresh fruits sold
pt['fresh fruits'].sum() # more than 55%

111199.0

In [49]:
# Check in how many orders was product yogurt sold
pt['yogurt'].sum()

52735.0

In [50]:
pt['milk'].sum()

48665.0

In [51]:
pt['fresh vegetables'].sum()

88872.0

# Creating association rules 5% support

In [59]:
# Apply the APRIORI algorithm to get frequent itemsets
# Rules supported in at least 5% of the transactions 
frequent_itemsets = apriori(pt, min_support=0.05, use_colnames=True)


In [60]:
frequent_itemsets

,support,itemsets
0,0.076635,(baking ingredients)
1,0.163865,(bread)
2,0.067765,(breakfast bakery)
3,0.074330,(butter)
4,0.069305,(candy chocolate)
...,...,...
151,0.051295,"(yogurt, milk, packaged vegetables fruits)"
152,0.051915,"(yogurt, packaged cheese, packaged vegetables ..."
153,0.062535,"(milk, fresh vegetables, fresh fruits, package..."
154,0.068325,"(packaged cheese, fresh vegetables, fresh frui..."


In [77]:
# Generate the association rules - by confidence
rulesConfidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.60)
rulesConfidence.sort_values(by='confidence', ascending=False, inplace=True)
rulesConfidence.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17,"(fresh fruits, fresh herbs)",(fresh vegetables),0.070135,0.444360,0.061815,0.881372,1.983463,0.030650,4.683872
41,"(yogurt, fresh vegetables, packaged vegetables...",(fresh fruits),0.087995,0.555995,0.076240,0.866413,1.558311,0.027315,3.323711
37,"(fresh vegetables, milk, packaged vegetables f...",(fresh fruits),0.073075,0.555995,0.062535,0.855765,1.539159,0.021906,3.078336
8,(fresh herbs),(fresh vegetables),0.093005,0.444360,0.078655,0.845707,1.903203,0.037327,3.601205
39,"(packaged cheese, fresh vegetables, packaged v...",(fresh fruits),0.081970,0.555995,0.068325,0.833537,1.499180,0.022750,2.667284


In [79]:
# Generate the association rules - by lift
rulesLift = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rulesLift.sort_values(by='lift', ascending=True, inplace=True)
rulesLift.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
110,(water seltzer sparkling water),(fresh vegetables),0.193005,0.444360,0.083355,0.431880,0.971915,-0.002409,0.978033
111,(fresh vegetables),(water seltzer sparkling water),0.444360,0.193005,0.083355,0.187584,0.971915,-0.002409,0.993328
232,"(fresh vegetables, fresh fruits)",(water seltzer sparkling water),0.317560,0.193005,0.063235,0.199128,1.031723,0.001944,1.007645
233,(water seltzer sparkling water),"(fresh vegetables, fresh fruits)",0.193005,0.317560,0.063235,0.327634,1.031723,0.001944,1.014983
82,(water seltzer sparkling water),(fresh fruits),0.193005,0.555995,0.111045,0.575348,1.034807,0.003735,1.045573


# Creating association rules 2.5% support

In [80]:
# Apply the APRIORI algorithm to get frequent itemsets
# Rules supported in at least 2.5% of the transactions 
frequent_itemsets = apriori(pt, min_support=0.025, use_colnames=True)


In [88]:
# Generate the association rules - by confidence
rulesConfidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rulesConfidence.sort_values(by='confidence', ascending=True, inplace=True)
rulesConfidence.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
172,"(chips pretzels, fresh fruits, packaged vegeta...",(fresh vegetables),0.057595,0.444360,0.040325,0.700148,1.575631,0.014732,1.853045
158,"(packaged cheese, fresh fruits, bread)",(fresh vegetables),0.044665,0.444360,0.031275,0.700213,1.575778,0.011428,1.853447
207,"(yogurt, packaged cheese, fresh fruits)",(fresh vegetables),0.070020,0.444360,0.049105,0.701300,1.578224,0.017991,1.860192
179,"(eggs, fresh fruits, milk)",(fresh vegetables),0.036475,0.444360,0.025580,0.701302,1.578230,0.009372,1.860208
15,(spices seasonings),(fresh vegetables),0.052165,0.444360,0.036660,0.702770,1.581533,0.013480,1.869394
4,(fresh dips tapenades),(fresh fruits),0.098205,0.555995,0.069040,0.703019,1.264434,0.014439,1.495063
230,"(yogurt, packaged cheese, fresh vegetables, fr...",(packaged vegetables fruits),0.049105,0.365415,0.034565,0.703900,1.926302,0.016621,2.143143
8,(hot cereal pancake mixes),(fresh fruits),0.045025,0.555995,0.031740,0.704942,1.267892,0.006706,1.504804
151,"(packaged vegetables fruits, lunch meat)",(fresh vegetables),0.053225,0.444360,0.037600,0.706435,1.589781,0.013949,1.892732
205,"(packaged cheese, fresh fruits, soy lactosefree)",(fresh vegetables),0.036390,0.444360,0.025735,0.707200,1.591502,0.009565,1.897676


In [89]:
# Generate the association rules - by lift
rulesLift = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rulesLift.sort_values(by='lift', ascending=True, inplace=True)
rulesLift.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
318,(fresh fruits),"(energy granola bars, packaged vegetables fruits)",0.555995,0.039500,0.032945,0.059254,1.500105,0.010983,1.020998
317,"(energy granola bars, packaged vegetables fruits)",(fresh fruits),0.039500,0.555995,0.032945,0.834051,1.500105,0.010983,2.675545
808,"(eggs, fresh vegetables, milk)",(fresh fruits),0.030660,0.555995,0.025580,0.834312,1.500574,0.008533,2.679762
821,(fresh fruits),"(eggs, fresh vegetables, milk)",0.555995,0.030660,0.025580,0.046008,1.500574,0.008533,1.016088
220,"(fresh fruits, crackers)",(milk),0.075480,0.243325,0.027570,0.365262,1.501129,0.009204,1.192107
221,(milk),"(fresh fruits, crackers)",0.243325,0.075480,0.027570,0.113305,1.501129,0.009204,1.042659
626,"(fresh vegetables, fresh fruits, packaged vege...",(bread),0.186580,0.163865,0.045910,0.246061,1.501606,0.015336,1.109022
639,(bread),"(fresh vegetables, fresh fruits, packaged vege...",0.163865,0.186580,0.045910,0.280170,1.501606,0.015336,1.130016
27,(cereal),(yogurt),0.092560,0.263675,0.036675,0.396229,1.502719,0.012269,1.219544
26,(yogurt),(cereal),0.263675,0.092560,0.036675,0.139092,1.502719,0.012269,1.054050


In [90]:
# Add a column with the length
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# Length=2 and Support>=0.2
frequent_itemsets[(frequent_itemsets['length'] == 2) & (frequent_itemsets['support'] >= 0.2)]

,support,itemsets,length
180,0.317560,"(fresh vegetables, fresh fruits)",2
200,0.269870,"(fresh fruits, packaged vegetables fruits)",2
236,0.234555,"(fresh vegetables, packaged vegetables fruits)",2


# Complementary Products

In [92]:
# High Confidence and high Lift 
rulesConfidence[(rulesConfidence['confidence'] >= 0.6) & (rulesConfidence['lift'] >= 1.6)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
172,"(chips pretzels, fresh fruits, packaged vegeta...",(fresh vegetables),0.057595,0.444360,0.040325,0.700148,1.575631,0.014732,1.853045
158,"(packaged cheese, fresh fruits, bread)",(fresh vegetables),0.044665,0.444360,0.031275,0.700213,1.575778,0.011428,1.853447
207,"(yogurt, packaged cheese, fresh fruits)",(fresh vegetables),0.070020,0.444360,0.049105,0.701300,1.578224,0.017991,1.860192
179,"(eggs, fresh fruits, milk)",(fresh vegetables),0.036475,0.444360,0.025580,0.701302,1.578230,0.009372,1.860208
15,(spices seasonings),(fresh vegetables),0.052165,0.444360,0.036660,0.702770,1.581533,0.013480,1.869394
...,...,...,...,...,...,...,...,...,...
226,"(packaged cheese, fresh vegetables, milk, pack...",(fresh fruits),0.032215,0.555995,0.028555,0.886388,1.594238,0.010644,3.908093
146,"(packaged vegetables fruits, fresh herbs)",(fresh vegetables),0.052535,0.444360,0.046980,0.894261,2.012470,0.023636,5.254824
231,"(yogurt, packaged cheese, fresh vegetables, pa...",(fresh fruits),0.038635,0.555995,0.034565,0.894655,1.609106,0.013084,4.214775
228,"(yogurt, fresh vegetables, milk, packaged vege...",(fresh fruits),0.036455,0.555995,0.032845,0.900974,1.620471,0.012576,4.483713


# Substitute Products


In [85]:
rulesLift2 = association_rules(frequent_itemsets, metric="lift", min_threshold=0.0)
rulesLift2.sort_values(by='lift', ascending=True, inplace=True)
rulesLift2.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
345,(fresh vegetables),(soft drinks),0.444360,0.087310,0.027845,0.062663,0.717709,-0.010952,0.973705
344,(soft drinks),(fresh vegetables),0.087310,0.444360,0.027845,0.318921,0.717709,-0.010952,0.815823
275,(fresh fruits),(soft drinks),0.555995,0.087310,0.039585,0.071197,0.815447,-0.008959,0.982652
274,(soft drinks),(fresh fruits),0.087310,0.555995,0.039585,0.453384,0.815447,-0.008959,0.812280
265,(paper goods),(fresh fruits),0.063575,0.555995,0.032090,0.504758,0.907847,-0.003257,0.896542
264,(fresh fruits),(paper goods),0.555995,0.063575,0.032090,0.057716,0.907847,-0.003257,0.993782
67,(fresh vegetables),(candy chocolate),0.444360,0.069305,0.028210,0.063485,0.916017,-0.002586,0.993785
66,(candy chocolate),(fresh vegetables),0.069305,0.444360,0.028210,0.407041,0.916017,-0.002586,0.937064
336,(fresh vegetables),(paper goods),0.444360,0.063575,0.025885,0.058252,0.916277,-0.002365,0.994348
337,(paper goods),(fresh vegetables),0.063575,0.444360,0.025885,0.407157,0.916277,-0.002365,0.937246


# Checking the department of different products

In [104]:
print(products[products['product_name']  == 'fresh fruits'])
print(products[products['product_name']  == 'fresh vegetables'])
print(products[products['product_name']  == 'packaged vegetables fruits'])

    product_id  department_id  product_name
81          24              4  fresh fruits
    product_id  department_id      product_name
35          83              4  fresh vegetables
    product_id  department_id                product_name
27         123              4  packaged vegetables fruits


In [108]:
print(products[products['product_name']  == 'yogurt'])
print(products[products['product_name']  == 'packaged cheese'])
print(products[products['product_name']  == 'milk'])

   product_id  department_id product_name
8         120             16       yogurt
    product_id  department_id     product_name
72          21             16  packaged cheese
     product_id  department_id product_name
108          84             16         milk


In [109]:
print(products[products['product_name']  == 'soft drinks'])

    product_id  department_id product_name
54          77              7  soft drinks


In [111]:
print(products[products['product_name']  == 'candy chocolate'])

    product_id  department_id     product_name
74          45             19  candy chocolate


In [112]:
print(products[products['product_name']  == 'paper goods'])

     product_id  department_id product_name
123          54             17  paper goods
